# Johdanto datatieteeseen 2021

#### Harjoitustyö - osa 2 - datan kerääminen

## Pirkanmaan omakotitalot

Tässä notebookissa esitetään Tampereen yliopiston Johdanto datatieteeseen -kurssin harjoitustyötä varten Etuovi.com -sivustolta tehty datan raavinta Pirkanmaan alueella myynnissä olevista omakotitaloista.

Haku Etuovi.com -sivustolla on tehty 24.4.2021

In [2]:
# Tarvittavat kirjastot
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
from requests import get
import pandas as pd

print('Importing done!')

Importing done!


In [3]:
# Datan raapimiseen tarvittavat taustatiedot
fileName = 'pirkanmaa_okt.csv'
headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
baseUrl = 'https://www.etuovi.com/myytavat-asunnot?haku=M1645601158&sivu='
html = get(baseUrl, headers=headers)
soup = bs(html.text, 'html.parser')

houseCards = soup.find_all('div', class_="ListPage__cardContainer__39dKQ")
tabs = soup.find_all('button', class_="theme__button__1YqFK theme__flat__13aFK theme__button__1YqFK theme__squared__17Uvn theme__neutral__1F1Jf Button__button__3K-jn Pagination__button__3H2wX")
max_tab = int(tabs[-1].contents[0])
print('Parameters set.')

Parameters set.


In [4]:
#Testataan yhteys sivustoon
print(html.text[:1000])


<!DOCTYPE html>
<html lang="fi">
    <head>
        <script>
            window.etuovi = {
                ad: {
                    qskvconsent: false,
                    placements: {}
                }
            }
        </script>
        <script>
        (function(){
            window.etuovi.cmpConfiguration = {
                ads: {
                    functional: [0]
                },
                facebookLogin: {
                    functional: [0]
                },
                giosgChat: {
                    functional: [0]
                },
                googleAnalytics: {
                    standard: [1, 8]
                },
                habitaChat: {
                    functional: [0]
                },
                nps: {
                    functional: [0]
                },
                recaptcha: {
                    functional: [0]
                },
                leadooChat: {
                    standard: [1],
                    spe

In [15]:
# Datan raavinta
data = {
'Alue': [],
'Kaupunki': [],
'Osoite': [],
'Hinta': [],
'Asuinpinta-ala': [],
'Kokonaispinta-ala': [],
'Yksikkö': [],
'Vuosi': [],
'Sauna': [],
'Huoneluku': [],
'Ominaisuudet': [],
}

# Loopataan kaikki tulosvälilehdet läpi
for i in range(max_tab):
    clear_output(wait=True)

    baseUrl = f'https://www.etuovi.com/myytavat-asunnot?haku=M1645601158&sivu={i+1}'
    html = get(baseUrl, headers=headers)
    soup = bs(html.text, 'html.parser')

    houseCards = soup.find_all('div', class_="ListPage__cardContainer__39dKQ")

    for temp in houseCards:
        spans = temp.find_all('span')

        isSize = False
        isPrice = False
        isYear = False

        # Raavitaan asunnon hinta, koko ja rakennusvuosi
        for item in spans:
            if isPrice:
                price = item.contents[0].replace('\xa0', '')[:-1]
                price = float(price.replace(',','.'))
                isPrice = False
            elif isSize:
                new_item = item.contents[0]
                if " / " in new_item:
                    size, whole_size_temp = new_item.split(' / ')
                    whole_size, unit = whole_size_temp.split(' ')
                    isSize = False
                else :
                    raw_size = item.contents[0]
                    try:
                        size, unit = raw_size.split(' ') 
                        whole_size = size
                    except ValueError:
                        print('Error!')
                    isSize = False
            elif isYear:
                try:
                    year = int(item.contents[0])
                except ValueError:
                    year = None
                isYear = False
            else:
                if 'Hinta' in item.contents and 'Hintamuutos' not in item.contents:
                    isPrice = True
                elif 'Koko' in item.contents:
                    isSize = True
                elif 'Vuosi' in item.contents:
                    isYear = True


        # Raavitaan osoitetiedot
        raw_address = temp.find('h4').contents[0]
        info = []
        info = raw_address.split(', ')
        for i in info:
            if i == '':  # Tarkistetaan, onko joukossa tyhjiä arvoja ja poistetaan tyhjät
                info.remove(i) 
        if len(info) == 3:
            address, area, city = info[0], info[1], info[2] # Osoite, Alue, Kaupunki
        elif len(info) == 2:
            address, city = raw_address.split(', ')
        else:
            address = raw_address

        # Raavitaan muut tiedot
        raw_type = temp.find('h5').contents[0]
        try:
            house_type, raw_attributes = raw_type.split(' | ')
            raw_attributes.replace(',', '+')
            attributes = raw_attributes.split(' + ') # Ominaisuudet
        except ValueError:
            house_type = raw_type

        # Lisätään tiedot muuttujiin
        data['Alue'].append(area.strip())
        data['Kaupunki'].append(city.strip())
        data['Osoite'].append(address.strip())
        data['Hinta'].append(price)
        data['Asuinpinta-ala'].append(size)
        data['Kokonaispinta-ala'].append(whole_size)
        data['Yksikkö'].append(unit)
        data['Vuosi'].append(year)
        data['Huoneluku'].append(attributes[0])

        # Tarkistetaan, onko talossa sauna
        if 's' in attributes:
            sauna = True
        else:
            sauna = False

        # Lisätään tieto saunasta, ja muut tiedot ominaisuuksiin
        data['Sauna'].append(sauna)
        data['Ominaisuudet'].append(attributes)
    
# Tallennetaan tiedot uuteen dataframeen
df = pd.DataFrame(data)
df.to_csv(f'{fileName}')
print('Done scraping')
print(f'{len(df)} rows loaded.')

# Tulostetaan dataframen tiedot
print("Here is the info of your dataframe:")
print(df.info())

# Tulostetaan dataframen ensimmäiset rivit
print("Here are the top rows of your dataframe:")
print(df.head())

Done scraping
297 rows loaded.
Here is the info of your dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 11 columns):
Alue                 297 non-null object
Kaupunki             297 non-null object
Osoite               297 non-null object
Hinta                297 non-null float64
Asuinpinta-ala       297 non-null object
Kokonaispinta-ala    297 non-null object
Yksikkö              297 non-null object
Vuosi                288 non-null float64
Sauna                297 non-null bool
Huoneluku            297 non-null object
Ominaisuudet         297 non-null object
dtypes: bool(1), float64(2), object(8)
memory usage: 23.6+ KB
None
Here are the top rows of your dataframe:
            Alue  Kaupunki                Osoite     Hinta Asuinpinta-ala  \
0      Metsäkylä  Ylöjärvi      Metsäkyläntie 31  210000.0            119   
1  Koivistonkylä   Tampere      Rintamäenkatu 30  365000.0            212   
2        Leinola   Tampere        Kale